In [1]:
using Revise
using GeometryBasics: Vec3f, Point3f
using LinearAlgebra
using GLMakie
using StaticArrays
using VMRobotControl
using VMRobotControl.Splines: CubicSpline
using DifferentialEquations
using MeshIO
include("../../functions.jl")

generate_q_init (generic function with 1 method)

## Importing ShadowHand URDF

### URDF Parsing

In [2]:
using FileIO, UUIDs
try
    FileIO.add_format(format"DAE", (), ".dae", [:DigitalAssetExchangeFormatIO => UUID("43182933-f65b-495a-9e05-4d939cea427d")])
catch
end

cfg = URDFParserConfig(;suppress_warnings=true) # This is just to hide warnings about unsupported URDF features
module_path = joinpath(splitpath(splitdir(pathof(VMRobotControl))[1])[1:end-1])
robot = parseURDF(joinpath(module_path, "URDFs/sr_description/sr_hand_vm_compatible.urdf"), cfg)

24DOF Mechanism{Float64} "shadowhand_motor" with 33 frames, 32 joints, 55 coordinates, 82 components

### Kinematic computations : Obtaining rail points

In [12]:
m = compile(robot)
kcache = new_kinematics_cache(m)  

rh_mftip_frame_id = get_compiled_frameID(m, "rh_mftip")
mftip_transform = get_transform(kcache, rh_mftip_frame_id)
display(mftip_transform.origin)

3-element SVector{3, Float64} with indices SOneTo(3):
  0.011
 -0.01
  0.44201000000000007

### Hand Visualization

In [212]:
# Target transform 
R = [1.0 0.0 0.0; 
     0.0 1.0 0.0;
     0.0 0.0 1.0]

t = [0.011, -0.075, 0.38]  # Example translation vector

# Construct the homogeneous transformation matrix
T = [R t; 0 0 0 1]

display(T)  # Show the matrix

4×4 Matrix{Float64}:
 1.0  0.0  0.0   0.011
 0.0  1.0  0.0  -0.075
 0.0  0.0  1.0   0.38
 0.0  0.0  0.0   1.0

In [16]:
# Compile robot model
m = compile(robot)
kcache = Observable(new_kinematics_cache(m))  # This stores the robot's joint state

# Create the figure
fig = Figure(size=(800, 600))

# Create interactive 3D scene
ls = LScene(fig[1, 1]; show_axis=true)  # 3D interactive scene

# Attach interactive camera controls
cam3d!(ls)  # Enables mouse interaction (rotate, zoom, pan)

# Plot the robot's visuals
robotvisualize!(ls, kcache)
display_frame(m, ls, "rh_ffmiddle")
display_frame(m, ls, "world")
#display_transform(ls, T)

# Display figure
display(fig)

GLMakie.Screen(...)

In [13]:
show(IOContext(stdout, :limit => false),  MIME("text/plain"), frames(robot))

33-element Vector{String}:
 "world"
 "rh_forearm"
 "rh_wrist"
 "rh_palm"
 "rh_manipulator"
 "rh_imu"
 "rh_ffknuckle"
 "rh_ffproximal"
 "rh_ffmiddle"
 "rh_ffdistal"
 "rh_fftip"
 "rh_mfknuckle"
 "rh_mfproximal"
 "rh_mfmiddle"
 "rh_mfdistal"
 "rh_mftip"
 "rh_rfknuckle"
 "rh_rfproximal"
 "rh_rfmiddle"
 "rh_rfdistal"
 "rh_rftip"
 "rh_lfmetacarpal"
 "rh_lfknuckle"
 "rh_lfproximal"
 "rh_lfmiddle"
 "rh_lfdistal"
 "rh_lftip"
 "rh_thbase"
 "rh_thproximal"
 "rh_thhub"
 "rh_thmiddle"
 "rh_thdistal"
 "rh_thtip"

## Creating the Virtual Mechanism System

### Gravity Compensation, Joint Limits and Joint Damping

In [3]:
# GRAVITY COMPENSATION

add_gravity_compensation!(robot, VMRobotControl.DEFAULT_GRAVITY)

# JOINT DAMPING and LIMIT SPRINGS

joint_limits = cfg.joint_limits

for joint_id in keys(joints(robot))
    limits = joint_limits[joint_id]
    isnothing(limits) && continue
    add_coordinate!(robot, JointSubspace(joint_id);  id="$(joint_id)_coord")
    @assert ~isnothing(limits.lower) && ~isnothing(limits.upper)
    add_deadzone_springs!(robot, 50.0, (limits.lower+0.1, limits.upper-0.1), "$(joint_id)_coord")
    add_component!(robot, LinearDamper(0.01, "$(joint_id)_coord"); id="$(joint_id)_damper")
end

### Creation of the rail

In [4]:
vm = Mechanism{Float64}("VirtualTracks")

track_points = Matrix{Float64}([0.011 -0.01 0.442; 0.011  -0.075  0.41 ; 0.011 -0.065 0.32; 0.011 -0.0275 0.28; 0.011 -0.01 0.34])
spline = CubicSpline(track_points)

add_frame!(vm; id = "cart_frame")
add_joint!(vm, Rail(spline, zero(Transform{Float64})); parent=root_frame(vm), child="cart_frame", id="RailJoint")
add_coordinate!(vm, FrameOrigin("cart_frame"); id="cart position")
add_coordinate!(vm, JointSubspace("RailJoint");  id="CartDistance")
add_coordinate!(vm, ConstCoord(0.0); id = "Cart target position")
add_coordinate!(vm, CoordDifference("CartDistance", "Cart target position"); id ="Cart position error")
add_component!(vm, LinearInerter(1.0, "cart position");  id="CartInertance") # Cart mass
add_component!(vm, LinearSpring(1000.0, "Cart position error"); id = "cart positioning spring")
add_component!(vm, LinearDamper(100.0, "CartDistance"); id="CartDamper");

### Addition of the multiples springs/dampers

In [ ]:
vms = VirtualMechanismSystem("myShadowVMS", robot, vm)
root = root_frame(vms.robot)

m = compile(robot)
kcache = new_kinematics_cache(m)  

cart_init_pos = SVector(0.011, -0.01, 0.442)

#Linking the four fingers to the same point, with non-zero equilibrium springs

K = SMatrix{3, 3}(100., 0., 0., 0., 100., 0., 0., 0., 100.)
D = SMatrix{3, 3}(10., 0., 0., 0., 10.0, 0., 0., 0., 10.)

#lightly constraining some joints to avoid unwanted motions
add_component!(vms, LinearSpring(10.0, ".robot.rh_LFJ5_coord"); id = "lf j5 angular spring")
add_component!(vms, LinearSpring(10.0, ".robot.rh_FFJ4_coord"); id = "ff j4 angular spring")
add_component!(vms, LinearSpring(10.0, ".robot.rh_MFJ4_coord"); id = "mf j4 angular spring")
add_component!(vms, LinearSpring(10.0, ".robot.rh_RFJ4_coord"); id = "rf j4 angular spring")
add_component!(vms, LinearSpring(10.0, ".robot.rh_LFJ4_coord"); id = "lf j4 angular spring")
add_component!(vms, LinearSpring(10.0, ".robot.rh_WRJ1_coord"); id = "wr j1 angular spring")

#Linking fingers to this target 
rh_fftip_frame_id = get_compiled_frameID(m, "rh_fftip")
fftip_transform = get_transform(kcache, rh_fftip_frame_id)
add_coordinate!(vms, FrameOrigin(".robot.rh_fftip"); id="ff position")
add_coordinate!(vms, FramePoint(".virtual_mechanism.cart_frame", fftip_transform.origin - cart_init_pos); id = "ff Target position") 
add_coordinate!(vms, CoordDifference("ff Target position", "ff position"); id="ff position error")

add_component!(vms, LinearSpring(K, "ff position error");           id="ff position spring")
add_component!(vms, LinearDamper(D, "ff position error");           id="ff position damper")

rh_mftip_frame_id = get_compiled_frameID(m, "rh_mftip")
mftip_transform = get_transform(kcache, rh_mftip_frame_id)
add_coordinate!(vms, FrameOrigin(".robot.rh_mftip"); id="mf position")
add_coordinate!(vms, FramePoint(".virtual_mechanism.cart_frame", mftip_transform.origin - cart_init_pos); id = "mf Target position") 
add_coordinate!(vms, CoordDifference("mf Target position", "mf position"); id="mf position error")

add_component!(vms, LinearSpring(K, "mf position error");           id="mf position spring")
add_component!(vms, LinearDamper(D, "mf position error");           id="mf position damper")

rh_rftip_frame_id = get_compiled_frameID(m, "rh_rftip")
rftip_transform = get_transform(kcache, rh_rftip_frame_id)
add_coordinate!(vms, FrameOrigin(".robot.rh_rftip"); id="rf position")
add_coordinate!(vms, FramePoint(".virtual_mechanism.cart_frame", rftip_transform.origin - cart_init_pos); id = "rf Target position") 
add_coordinate!(vms, CoordDifference("rf Target position", "rf position"); id="rf position error")

add_component!(vms, LinearSpring(K, "rf position error");           id="rf position spring")
add_component!(vms, LinearDamper(D, "rf position error");           id="rf position damper")

rh_lftip_frame_id = get_compiled_frameID(m, "rh_lftip")
lftip_transform = get_transform(kcache, rh_lftip_frame_id)
add_coordinate!(vms, FrameOrigin(".robot.rh_lftip"); id="lf position")
add_coordinate!(vms, FramePoint(".virtual_mechanism.cart_frame", lftip_transform.origin - cart_init_pos); id = "lf Target position") 
add_coordinate!(vms, CoordDifference("lf Target position", "lf position"); id="lf position error")

add_component!(vms, LinearSpring(K, "lf position error");           id="lf position spring")
add_component!(vms, LinearDamper(D, "lf position error");           id="lf position damper")

# THUMB HANDLING 
add_coordinate!(vms, FrameOrigin(".robot.rh_thtip"); id ="th position")
add_coordinate!(vms, FrameOrigin(".robot.rh_ffmiddle"); id= "ff middle position")
add_coordinate!(vms, CoordDifference("th position", "ff middle position"); id = "th ff dist")
add_coordinate!(vms, CoordNorm("th ff dist"); id="th ff norm")
add_coordinate!(vms, ConstCoord(0.15); id="th spring length")
add_coordinate!(vms, CoordDifference("th ff norm", "th spring length"); id = "th position error")

add_component!(vms, LinearSpring(100.0, "th position error"); id="th position spring")
add_component!(vms, LinearDamper(100.0, "th position error"); id="th position damper")

add_coordinate!(vms, ConstCoord(1.22); id="th j5 target angle")
add_coordinate!(vms, JointSubspace(".robot.rh_THJ5"); id= "th j5 angle") # --> Should be J4 and not j5 --> to modify
add_coordinate!(vms, CoordDifference("th j5 target angle", "th j5 angle"); id="th j5 angle error")

add_component!(vms, LinearSpring(100.0, "th j5 angle error"); id="th j5 angular spring")
add_component!(vms, LinearDamper(10.0, "th j5 angle error"); id="th j5 angular damper")


"th j5 angular damper"

## Simulating the Robot

### Setting Up the Simulation

In [6]:
using Logging: global_logger
using TerminalLoggers: TerminalLogger
global_logger(TerminalLogger())

"""
Update the vms based on the "coord", which is a coordinate varying between 
0 and 1, 0 being the starting position of the medium wrap motion while 1 being 
the end
"""
function update_medium_wrap_coord(args, cache, coord)
    target_rail_id, th_spring_length_id, th_j5_target_angle_id = args
    # target_rail_id = args

    #update the cart position on the track
    rail_min = 0
    rail_max = 0.25
    rail_value = rail_min + (rail_max - rail_min)*coord # linear proportion
    cache[target_rail_id] = remake(cache[target_rail_id] ; coord_data=ConstCoord(rail_value))

    #update the length of the spring between the thumb and the ff finger
    length_max = 0.12
    length_min = 0.0
    length_value = length_max - (length_max - length_min)*coord 
    cache[th_spring_length_id] = remake(cache[th_spring_length_id] ; coord_data = ConstCoord(length_value))

    #update the angle of the thumb
    angle_max = 1.22
    angle_min = 0.0
    angle_value = angle_max - (angle_max - angle_min)*coord
    cache[th_j5_target_angle_id] = remake(cache[th_j5_target_angle_id] ; coord_data = ConstCoord(angle_value))

    nothing
end

function f_setup(cache) 
    return (get_compiled_coordID(cache, ".virtual_mechanism.Cart target position") , get_compiled_coordID(cache, "th spring length"), get_compiled_coordID(cache, "th j5 target angle"))
end

function f_control(cache, t, args, extra)

    t_end = 8. #we want to reach the end position at t = t_end 
    
    if t <= t_end 
        coord_value = t/t_end
        update_medium_wrap_coord(args, cache, coord_value)
    end
end

tspan = (0., 10.)
vms_compiled = compile(vms)

q = (zero_q(vms_compiled.robot), zero_q(vms_compiled.virtual_mechanism)) # Robot joint angle, vm joint angles
q̇ = (zero_q̇(vms_compiled.robot), zero_q̇(vms_compiled.virtual_mechanism)) # Robot joint velocity, vm joint velocities

g = VMRobotControl.DEFAULT_GRAVITY
dcache = new_dynamics_cache(vms_compiled)
prob = get_ode_problem(dcache, g, q, q̇, tspan; f_setup, f_control)
@info "Simulating shadow robot with medium wrap"

sol = solve(prob, Tsit5(), progress=true; maxiters=1e6, abstol=1e-5, reltol=1e-5);

[ Info: Simulating shadow robot with medium wrap
ODE   0%|█                                              |  ETA: N/A
ODE   0%|█                                              |  ETA: 2:40:35
ODE   0%|█                                              |  ETA: 1:51:58
ODE   0%|█                                              |  ETA: 1:34:12
ODE   0%|█                                              |  ETA: 1:26:32
ODE   0%|█                                              |  ETA: 1:18:58
ODE   0%|█                                              |  ETA: 1:13:42
ODE   0%|█                                              |  ETA: 1:08:29
ODE   0%|█                                              |  ETA: 1:04:10
ODE   0%|█                                              |  ETA: 1:00:32
ODE   0%|█                                              |  ETA: 0:57:14
ODE   0%|█                                              |  ETA: 0:54:35
ODE   0%|█                                              |  ETA: 0:52:01
ODE   0%|█         

### Visualizing the Results

In [7]:
fig = Figure(; size=(720, 720), figure_padding=0)
display(fig)
ls = LScene(fig[1, 1]; show_axis=false)
cam = cam3d!(ls; center=true)
cam.lookat[] = [0.025, 0., 0.24]
cam.eyeposition[] = [-0.25, -0.6, 0.62]

plotting_t = Observable(0.0)
plotting_kcache = Observable(new_kinematics_cache(compile(vms)))
robotvisualize!(ls, plotting_kcache)

plotting_vm_kcache = map(plotting_kcache) do k
    VMRobotControl.virtual_mechanism_cache(k)
end
cartID = get_compiled_coordID(plotting_kcache[], ".virtual_mechanism.cart position")
scatter!(ls, plotting_kcache, cartID; color=:red, marker=:rect, markersize=5)
robotsketch!(ls, plotting_vm_kcache; scale = 0.1)



savepath = joinpath(module_path, "docs/src/assets/shadowhand_medium_wrap.mp4")
display(fig)
animate_robot_odesolution(fig, sol, plotting_kcache, savepath; t=plotting_t);